# Run a full mission of basestation files


In [ ]:
import sys
import importlib
sys.path.append('/Users/eddifying/Cloudfree/gitlab-cloudfree/seagliderOG1')
import warnings
warnings.simplefilter("ignore", category=Warning)

In [ ]:
from seagliderOG1 import readers
from seagliderOG1 import plotters
from seagliderOG1 import convertOG1
from seagliderOG1 import vocabularies
import xarray as xr
import os

In [ ]:
importlib.reload(readers)
importlib.reload(convertOG1)
#input_loc = "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/015/20040924/"
input_loc = "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/014/20040924/"
input_loc = "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/004/20031002/"
input_loc = "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/008/20031002/"
input_loc =  "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20050406/"

def process_and_save_data(input_location, save=False, output_dir='../data'):
    # Load and concatenate all datasets from the server
    list_datasets = readers.read_basestation(input_location)
    
    # Convert the list of datasets to OG1
    ds_all = convertOG1.convert_to_OG1(list_datasets)
    
    if save:
        # Generate the output file path
        # Need to get the serial number of the platform for this
#        output_file = os.path.join(output_dir, id_str + '_' + ds_all.start_date + '_delayed.nc')
        output_file = os.path.join('../data', ds_all.attrs['id'] + '.nc')

        # Check if the file exists and delete it if it does
        if os.path.exists(output_file):
            user_input = input(f"File {output_file} already exists. Do you want to delete it? (yes/no): ")
            if user_input.lower() != 'yes':
                print("File not deleted. Exiting the process.")
                return ds_all
            os.remove(output_file)
        
        # Save the dataset to a NetCDF file
        convertOG1.save_dataset(ds_all, output_file)
    
    return ds_all

# Example usage
ds_all = process_and_save_data(input_loc, save=True)

In [ ]:
importlib.reload(convertOG1)
input_loc =  "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20050406/"
save=True
# Troubleshooting - run outside a function
# Load and concatenate all datasets from the server
list_datasets = readers.read_basestation(input_loc)

datasets = list_datasets
# Convert the list of datasets to OG1
processed_datasets = []
for ds in datasets:
    ds_new, attr_warnings, sg_cal, dc_other, dc_log = convertOG1.convert_to_OG1_dataset(ds)
    processed_datasets.append(ds_new)

concatenated_ds = xr.concat(processed_datasets, dim='N_MEASUREMENTS')
concatenated_ds = concatenated_ds.sortby('TIME')

# Apply attributes
ordered_attributes = update_dataset_attributes(datasets[0], contrib_to_append)
for key, value in ordered_attributes.items():
    concatenated_ds.attrs[key] = value

# Construct the platform serial number
PLATFORM_SERIAL_NUMBER = 'sg' + concatenated_ds.attrs['id'][1:4]
print(PLATFORM_SERIAL_NUMBER)
concatenated_ds['PLATFORM_SERIAL_NUMBER'] = PLATFORM_SERIAL_NUMBER
concatenated_ds['PLATFORM_SERIAL_NUMBER'].attrs['long_name'] = "glider serial number"

# Construct the unique identifier attribute
id = f"{PLATFORM_SERIAL_NUMBER}_{concatenated_ds.start_date}_delayed"
concatenated_ds.attrs['id'] = id

if save:
    # Generate the output file path
    # Need to get the serial number of the platform for this
#        output_file = os.path.join(output_dir, id_str + '_' + ds_all.start_date + '_delayed.nc')
    output_file = os.path.join('../data', concatenated_ds.attrs['id'] + '.nc')

    # Check if the file exists and delete it if it does
    if os.path.exists(output_file):
        user_input = input(f"File {output_file} already exists. Do you want to delete it? (yes/no): ")
        if user_input.lower() != 'yes':
            print("File not deleted. Exiting the process.")
            return ds_all
        os.remove(output_file)
    
    # Save the dataset to a NetCDF file
    convertOG1.save_dataset(concatenated_ds, output_file)

In [ ]:
processed_datasets
coordinates_dict = {i: list(ds.coords) for i, ds in enumerate(processed_datasets)}
coordinates_dict

missing_longitude = [i for i, coords in coordinates_dict.items() if 'LONGITUDE' not in coords]
missing_longitude

datasets_with_longitude = [i for i, coords in coordinates_dict.items() if 'LONGITUDE' in coords]
datasets_with_longitude

import matplotlib.pyplot as plt


In [ ]:
print(list(ds.variables.keys()))
print(datasets[0])


In [ ]:
concatenated_ds = xr.concat(processed_datasets, dim='N_MEASUREMENTS')
concatenated_ds = concatenated_ds.sortby('TIME')
# Generate the output file path
# Need to get the serial number of the platform for this
#        output_file = os.path.join(output_dir, id_str + '_' + ds_all.start_date + '_delayed.nc')
output_file = os.path.join('../data', concatenated_ds.attrs['id'] + '.nc')

# Check if the file exists and delete it if it does
if os.path.exists(output_file):
    user_input = input(f"File {output_file} already exists. Do you want to delete it? (yes/no): ")
    if user_input.lower() != 'yes':
        print("File not deleted. Exiting the process.")
        return concatenated_ds
    os.remove(output_file)

# Save the dataset to a NetCDF file
convertOG1.save_dataset(concatenated_ds, output_file)